# Example of a transport-limited LEM using `AreaSlopeTransporter`

*(Greg Tucker, University of Colorado Boulder, October 2022)*

This tutorial notebook illustrates how to use the Landlab `AreaSlopeTransporter` component to create a simple transport-limited Landscape Evolution Model (LEM).

## Background

The `AreaSlopeTransporter` is a landscape evolution component that models the time rate of change of elevation at a set of grid nodes, each of which has a defined contributing drainage area $A$ (field `drainage_area`) and a local steepest-descent slope gradient, $S$, defined from the node itself toward one of its neighboring nodes. The drainage area and slope can be computed with a drainage-routing component such as `FlowAccumulator` or `PriorityFloodFlowAccumulator`. The component is designed to function as an integral part of a transport-limited landscape evolution model in the spirit of Willgoose et al. (1991a,b,c,d).

Once $A$ and $S$ have been calculated, the volumetric outgoing sediment flux at each grid node is calculated using

$$Q_{out} = k_s A^m S^n$$

The component's default parameters are $m=1.4$, $n=2.1$, and $k_s = 0.0055$ (with time units of years). These defaults are based on Willgoose et al. (1991a), who used the Einstein-Brown sediment transport formula as a basis, but here modified to include an empirical sub-grid-cell channel width. The derivation is provided in a separate notebook (see *einstein-brown.ipynb*). The essence of the odd-seeming exponent values is this: Einstein-Brown describes a cubic dependence on shear stress, which essentially has a 2/3 power dependence on slope gradient. That gets you the power 2; the extra 0.1 shows up because of the use of the Manning equation to describe roughness, in which the roughness factor has a slight depenence on flow depth. The 1.4 exponent on drainage area has a similar story, with one difference from Willgoose et al. (1991a) (who used a 1.8 power) being the assumption that bankfull width is proportional to the square root of discharge. The derivation in the *einstein-brown.ipynb* notebook describes how that leads to 1.4 power scaling. Note that Howard (1994) also used Einstein-Brown as the basis for an alluvial-channel element of an otherwise detachment-limited landscape evolution model.

The time rate of change of elevation is calculated over a grid cell at node $i$ with surface area $\Lambda_i$, such that mass continuity gives the time rate of elevation ($z_i$) change (in the absence of other processes) as:

$$\frac{dz_i}{dt} = \frac{Q_{in} - Q_{out}}{\Lambda_i}$$

The sediment influx at each node is calculated as the sum of outflux of all upstream neighbor nodes that flow to it.

Note that this is a no-frills component, at least as of this writing. It uses only drainage area (not discharge, even if calculated). It uses a very basic forward-Euler solution algorithm, and so it requires very small time steps. There is no adaptive solver. It is designed mainly for illustrative purposes: to show how a basic transport-limited area-slope LEM behaves. As with all Landlab components, the user should understand the theory and be familiar with the background literature!

*This Landlab component is dedicated to the memory of two remarkable hydrologists: Garry Willgoose and Ignacio Rodriguez-Iturbe.*

## Example

The example uses a very small (15 x 15) domain, initially uplifted 25 m above the baselevel, which is represented by a single node in one corner of the grid. The small domain and short run duration are meant to keep the total run time under a second or so, depending on the computer architecture. The user is encouraged to experiment with larger domains and longer run durations, but beware that in general the larger the domain, the smaller the time-step size will need to be to ensure numerical stability. (If the concept of numerical instability is unfamiliar, then consultation of a textbook and/or online resources on numerical methods for partial differential equations is highly recommended.)

In [ ]:
import time

import matplotlib.pyplot as plt
import numpy as np

from landlab import RasterModelGrid, imshow_grid
from landlab.components import AreaSlopeTransporter, FlowAccumulator

In [ ]:
# Parameters
nrows = 15  # number of node rows
ncols = 15  # number of node columns
dx = 20.0  # grid node spacing, m
run_duration = 4000.0  # run duration, y
dt = 2.5  # time-step duration, y
noise_amplitude = 1.0  # amplitude of initial random noise, m
initial_height = 25.0  # initial plateau height, m
seed = 1  # random seed

In [ ]:
# Control and derived parameters, and other setup
elapsed_time = 0.0
np.random.seed(seed)

In [ ]:
# Initialize grid and set boundary conditions
grid = RasterModelGrid((nrows, ncols), xy_spacing=dx)
grid.status_at_node[grid.perimeter_nodes] = grid.BC_NODE_IS_CLOSED
grid.status_at_node[ncols - 1] = grid.BC_NODE_IS_FIXED_VALUE  # corner outlet

In [ ]:
# Create and initialize elevation field
elev = grid.add_zeros("topographic__elevation", at="node")
elev[grid.core_nodes] = initial_height + noise_amplitude * np.random.rand(
    grid.number_of_core_nodes
)

In [ ]:
# Instantiate components
fa = FlowAccumulator(grid, flow_director="FlowDirectorD8")
ast = AreaSlopeTransporter(grid)

In [ ]:
# Main loop
start = time.time()
while elapsed_time < run_duration:
    fa.run_one_step()
    ast.run_one_step(dt)
    elapsed_time += dt

wall_time = time.time() - start
print("Wall time for run:", wall_time, "s")

In [ ]:
# Display the final topography in map view
imshow_grid(grid, elev)

In [ ]:
# Display the final topography in 3d surface view
X = np.arange(0, dx * ncols, dx)
Y = np.arange(0, dx * nrows, dx)
X, Y = np.meshgrid(X, Y)

fig, ax = plt.subplots(subplot_kw={"projection": "3d"})
ax.plot_surface(X, Y, elev.reshape((nrows, ncols)))

## References

Howard, A. D. (1994). A detachment‐limited model of drainage basin evolution. Water resources research, 30(7), 2261-2285.

Tucker, G. E., & Hancock, G. R. (2010). Modelling landscape evolution. Earth Surface Processes and Landforms, 35(1), 28-50.

Willgoose, G. (2018). *Principles of soilscape and landscape evolution.* Cambridge University Press.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991a). A coupled channel network growth and hillslope evolution model: 1. Theory. Water Resources Research, 27(7), 1671-1684.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991b). A coupled channel network growth and hillslope evolution model: 2. Nondimensionalization and applications. Water Resources Research, 27(7), 1685-1696.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991c). A physical explanation of an observed link area‐slope relationship. Water Resources Research, 27(7), 1697-1702.

Willgoose, G., Bras, R. L., & Rodriguez‐Iturbe, I. (1991d). Results from a new model of river basin evolution. Earth Surface Processes and Landforms, 16(3), 237-254.
